# Binance Client Test Notebook

In [ ]:
import sys
import os
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add the src directory to the Python path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from clients.binance_client import BinanceClient

## Initialize Binance Client

In [ ]:
# Initialize client
client = BinanceClient(api_key=os.getenv('binance_api_key'), secret=os.getenv('binance_api_secret'))

## Fetch Current Funding Rates

In [ ]:
symbols = ['BTC/USDT:USDT', 
           'ETH/USDT:USDT', 
           'SOL/USDT:USDT', 
           'XRP/USDT:USDT', 
           'BNB/USDT:USDT', 
           'DOGE/USDT:USDT', 
           '1000PEPE/USDT:USDT', 
           'TON/USDT:USDT', 
           'AVAX/USDT:USDT', 
           'WIF/USDT:USDT', 
           'MATIC/USDT:USDT', 
           'AAVE/USDT:USDT', 
           'LINK/USDT:USDT', 
           'SUI/USDT:USDT', 
           'ADA/USDT:USDT']
current_rates = client.get_current_funding_rates(symbols)
print("Current Funding Rates:")
display(current_rates)

## Visualize Current Funding Rates

In [ ]:
plt.figure(figsize=(10, 6))
current_rates_sorted = current_rates.sort_values(by='Annualized Funding Rate')
sns.barplot(x='Symbol', y='Annualized Funding Rate', data=current_rates_sorted)
plt.title('Current Annualized Funding Rates')
plt.xticks(rotation=45)
plt.show()

## Fetch current positions and prices on Future and Spot

In [ ]:
# Fetch spot position
future_symbol = 'LINK/USDT:USDT'
spot_symbol = 'LINK/USDT'
spot_position = client.fetch_spot_position(spot_symbol)
print(f"Spot position for {spot_symbol}: {spot_position}")

# Fetch futures position
futures_position = client.fetch_futures_position(future_symbol)
print(f"Futures position for {future_symbol}:")
display(futures_position)

# Fetch real-time prices
real_time_prices = client.fetch_real_time_prices(spot_symbol)
print(f"Real-time prices for {spot_symbol}:")
display(real_time_prices)

## Get OHLCV data

In [ ]:
symbol = 'BTC/USDT:USDT'
start_time = datetime.now() - timedelta(days=30)  # Last 30 days
ohlcv_data = client.get_ohlcv(symbol, timeframe='1d', since=start_time, limit=30)
display(ohlcv_data)

## Fetch Historical Funding Rates

In [ ]:
start_time = datetime.now() - timedelta(days=30)
start_time.timestamp() * 1000

In [ ]:
# Fetch Historical Funding Rates
symbol = 'BTC/USDT:USDT'
# Define start_time as a datetime object
start_time = datetime.now() - timedelta(days=30)
# Define end_time as the current datetime
end_time = datetime.now()

# Pass both datetime objects to the method
historical_rates = client.get_historical_funding_rates(symbol, start_time, end_time)
print("Historical Funding Rates:")
display(historical_rates.head())

## Visualize Historical Funding Rates

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(historical_rates['datetime'], historical_rates['annualized_rate'])
plt.title(f'Historical Annualized Funding Rates for {symbol}')
plt.xlabel('Date')
plt.ylabel('Annualized Funding Rate (%)')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

## Fetch OHLCV Data

In [ ]:
ohlcv_data = client.get_ohlcv(symbol, '1d', since=start_time)
print("OHLCV Data:")
display(ohlcv_data.head())

## Visualize OHLCV Data

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(ohlcv_data['datetime'], ohlcv_data['close'])
plt.title(f'Closing Prices for {symbol}')
plt.xlabel('Date')
plt.ylabel('Price (USDT)')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

## Analyze Correlation Between Price and Funding Rate

In [ ]:
# Merge OHLCV and funding rate data
merged_data = pd.merge(ohlcv_data, historical_rates, left_on='datetime', right_on='datetime', how='inner')

# Calculate correlation
correlation = merged_data['close'].corr(merged_data['annualized_rate'])
print(f"Correlation between price and funding rate: {correlation:.2f}")

# Visualize
plt.figure(figsize=(10, 6))
plt.scatter(merged_data['close'], merged_data['annualized_rate'])
plt.title(f'Price vs Funding Rate for {symbol}')
plt.xlabel('Price (USDT)')
plt.ylabel('Annualized Funding Rate (%)')
plt.grid(True)
plt.show()